In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [135]:
#Import the datasets from https://www.england.nhs.uk/statistics/statistical-work-areas/diagnostics-waiting-times-and-activity/diagnostics-census-data/
#I have combined the quarterly data into yearly csv files. 

#Each dataframe contains, for each quarter, the number of patients in the NHS which waited between 6 & 13 weeks and 
#the patients which waited over 13 weeks to recieve diagnostic treatment via consultant for different area's of medicine.
wait_time_2017_18=pd.read_csv("2017-18.csv")
wait_time_2016_17=pd.read_csv("2016-17.csv")
wait_time_2015_16=pd.read_csv("2015-16.csv")
wait_time_2014_15=pd.read_csv("2014-15.csv")
wait_time_2013_14=pd.read_csv("2013-14.csv")
wait_time_2012_13=pd.read_csv("2012-13.csv")
wait_time_2011_12=pd.read_csv("2011-12.csv")
wait_time_2010_11=pd.read_csv("2010-11.csv")
wait_time_2009_10=pd.read_csv("2009-10.csv")
wait_time_2008_09=pd.read_csv("2008-09.csv")
wait_time_2007_08=pd.read_csv("2007-08.csv")
#List containing each dataframe.
wait_time_list=[wait_time_2017_18,wait_time_2016_17,wait_time_2015_16,wait_time_2014_15,wait_time_2013_14,wait_time_2012_13,wait_time_2011_12,wait_time_2010_11,wait_time_2009_10,wait_time_2008_09,wait_time_2007_08]

#Removing the empty row at the bottom of some dataframes.
for i in range(0,len(wait_time_list)):
    if wait_time_list[i].shape[0]>13:
        wait_time_list[i]=wait_time_list[i].drop(13)
               
    else:
        continue
#Check to see if all dataframes have the correct number of rows
for item in wait_time_list:
    if item.shape[0] != 13:
        print("Error, the table has too many columns")
    else:
        continue

#Rename columns of each dataframe, drop the repeated columns.
for i in range(0,len(wait_time_list)):
    if len(wait_time_list[i].columns)==12:
        wait_time_list[i].columns=["Department","06 < 13 Weeks, Q1","13+ Weeks, Q1","Area","06 < 13 Weeks, Q2","13+ Weeks, Q2","Area","06 < 13 Weeks, Q3","13+ Weeks, Q3","Area","06 < 13 Weeks, Q4","13+ Weeks, Q4"]
    if "Area" in wait_time_list[i].columns.values:
        wait_time_list[i]=wait_time_list[i].drop("Area", axis=1)
    if "Area" in list(wait_time_list[i].iloc[0]):
        wait_time_list[i]=wait_time_list[i].drop(0,axis=0)
        


for i in range(0,len(wait_time_list)):
    
    wait_time_list[i].iloc[:,wait_time_list[i].columns != "Department"]=wait_time_list[i].iloc[:,wait_time_list[i].columns != "Department"].apply(lambda x: x.str.replace(",","").astype(float))        
    
    #TRYING TO SORT THIS BIT ---> #wait_time_list[i].loc[:,"total"]=wait_time_list[i].append(wait_time_list[i].sum().transpose()) 
#Iterate through list to attach a row to each dataframe consisting of the year and remove the row totals. We'll convert these to column totals.

for i in range(0,len(wait_time_list)):
    years=[2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007]
    wait_time_list[i].insert(0, "Year",[years[i],years[i],years[i],years[i],years[i],years[i],years[i],years[i],years[i],years[i],years[i],years[i]])
    wait_time_list[i]=wait_time_list[i].drop(12)
    
    
#Rename the individual data frames to reflect the changes in shape and added info.
wait_time_2017_18=wait_time_list[0]
wait_time_2016_17=wait_time_list[1]
wait_time_2015_16=wait_time_list[2]
wait_time_2014_15=wait_time_list[3]
wait_time_2013_14=wait_time_list[4]
wait_time_2012_13=wait_time_list[5]
wait_time_2011_12=wait_time_list[6]
wait_time_2010_11=wait_time_list[7]
wait_time_2009_10=wait_time_list[8]
wait_time_2008_09=wait_time_list[9]
wait_time_2007_08=wait_time_list[10]


#Function that we will use to put dataframes into one big super dataframe. To do this properly, will need to turn department row into column headings.
wait_time_all=pd.concat([wait_time_2017_18,wait_time_2016_17,wait_time_2015_16,wait_time_2014_15,wait_time_2013_14,wait_time_2012_13,wait_time_2011_12,wait_time_2010_11,wait_time_2009_10,wait_time_2008_09,wait_time_2007_08], axis=0)
#wait_time_all=wait_time_all.drop(12)

TypeError: Can only append a Series if ignore_index=True or if the Series has a name

In [134]:
wait_time_2009_10.sum(axis=0).transpose()

Year                                                             22099
Department           AudiologyCardiologyEndoscopyGI PhysiologyImagi...
06 < 13 Weeks, Q1                                               8457.0
13+ Weeks, Q1                                                   1205.0
06 < 13 Weeks, Q2                                               6853.0
13+ Weeks, Q2                                                    859.0
06 < 13 Weeks, Q3                                               8207.0
13+ Weeks, Q3                                                    720.0
06 < 13 Weeks, Q4                                               6296.0
13+ Weeks, Q4                                                    892.0
total                                                                0
dtype: object

In [128]:
wait_time_all.head(30)

,Year,Department,"06 < 13 Weeks, Q1","13+ Weeks, Q1","06 < 13 Weeks, Q2","13+ Weeks, Q2","06 < 13 Weeks, Q3","13+ Weeks, Q3","06 < 13 Weeks, Q4","13+ Weeks, Q4",total
1,2017,Audiology,832.0,128.0,1546.0,276.0,1439.0,249.0,834.0,139.0,NaN
2,2017,Cardiology,4733.0,1296.0,4392.0,1275.0,5119.0,1197.0,4647.0,1396.0,NaN
3,2017,Endoscopy,5224.0,1686.0,5723.0,1851.0,5670.0,1874.0,5609.0,1362.0,NaN
4,2017,GI Physiology,487.0,365.0,336.0,499.0,353.0,492.0,538.0,560.0,NaN
5,2017,Imaging,8097.0,1116.0,7927.0,1051.0,9796.0,1177.0,12218.0,1949.0,NaN
6,2017,Neurophysiology,806.0,315.0,878.0,279.0,812.0,227.0,949.0,369.0,NaN
7,2017,Ophthalmic Science,270.0,122.0,317.0,167.0,359.0,239.0,373.0,170.0,NaN
8,2017,Pathology,1411.0,705.0,2664.0,755.0,3424.0,565.0,4187.0,722.0,NaN
9,2017,Respiratory Physiology,626.0,280.0,741.0,320.0,1014.0,547.0,1551.0,591.0,NaN
10,2017,Urodynamics,311.0,94.0,259.0,84.0,307.0,53.0,304.0,45.0,NaN


In [6]:
#Lets try and plot one of the columns and see how it goes.


SyntaxError: invalid syntax (Temp/ipykernel_10196/2732634680.py, line 2)